# Basketball Jersey Numbers OCR - Google Colab

Sistema de deteccion de numeros en camisetas de baloncesto usando YOLOv8

**Modelo**: basketball-jersey-numbers-ocr/7  
**GPU**: NVIDIA Tesla T4  
**Inferencia**: 100% local (costo cero)  

## Configuracion del Runtime

**IMPORTANTE**: Antes de ejecutar, verifica que el runtime tenga GPU:

1. Runtime > Change runtime type
2. Hardware accelerator: **GPU (T4)**
3. Save

In [ ]:
# CELDA 1: Instalacion de dependencias
# Ejecutar esta celda primero (tarda ~2-3 minutos)

print("Instalando dependencias para Basketball Jersey OCR...")
print("Este proceso puede tardar 2-3 minutos\n")

# Instalar dependencias principales
!pip install -q inference supervision gradio roboflow pillow

print("\n[OK] Dependencias instaladas correctamente")
print("Nota: Los warnings de compatibilidad de pandas/cryptography son normales y no afectan el funcionamiento")

In [ ]:
# CELDA 2: Verificacion de GPU
# Confirma que estas usando GPU Tesla T4

import torch

print("=" * 70)
print("VERIFICACION DE GPU")
print("=" * 70)

if torch.cuda.is_available():
    print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Memoria total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Memoria asignada: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"Memoria en cache: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")
    print("\n[OK] GPU detectada correctamente")
else:
    print("[ERROR] GPU no detectada")
    print("Verifica: Runtime > Change runtime type > GPU")

print("=" * 70)

In [ ]:
# CELDA 3: Importaciones

import os
import csv
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Tuple

import numpy as np
import gradio as gr
from PIL import Image

print("[OK] Modulos importados correctamente")

In [ ]:
# CELDA 4: Clase JerseyAnalyzer
# Analizador de dorsales con inferencia local en GPU

class JerseyAnalyzer:
    """Analizador de numeros de camisetas de baloncesto con inferencia local"""

    def __init__(self, api_key: str, model_id: str = "basketball-jersey-numbers-ocr/7"):
        """
        Inicializa el analizador con inferencia local en GPU

        Args:
            api_key: Roboflow API key (solo para descargar modelo)
            model_id: ID del modelo en formato workspace/project/version
        """
        self.api_key = api_key
        self.model_id = model_id
        self.model = None
        
        # Usar rutas de Colab (/content/)
        self.output_dir = Path("/content/outputs/detections")
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.csv_log = Path("/content/jersey_log.csv")

        # Inicializar CSV si no existe
        if not self.csv_log.exists():
            with open(self.csv_log, 'w', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow(['Timestamp', 'Numero Detectado', 'Confianza', 'Archivo'])

        self._cargar_modelo()

    def _cargar_modelo(self):
        """Carga el modelo para inferencia local en GPU"""
        try:
            print(f"\nCargando modelo {self.model_id} para inferencia local...")
            from inference import get_model

            # Inicializar modelo con API key (solo descarga, no consume creditos)
            self.model = get_model(
                model_id=self.model_id,
                api_key=self.api_key
            )
            print(f"[OK] Modelo cargado en GPU local")

        except Exception as e:
            print(f"[ERROR] Error al cargar modelo: {e}")
            raise

    def detectar_numeros(
        self,
        imagen: np.ndarray,
        confianza_min: float = 0.4
    ) -> Tuple[np.ndarray, List[Dict]]:
        """
        Detecta numeros en camiseta con inferencia local

        Args:
            imagen: Imagen en formato numpy array (RGB)
            confianza_min: Umbral minimo de confianza (0.0-1.0)

        Returns:
            Tupla de (imagen_anotada, lista_detecciones)
        """
        if self.model is None:
            raise RuntimeError("Modelo no inicializado")

        # Inferencia local (NO consume creditos de API)
        resultados = self.model.infer(imagen, confidence=confianza_min)[0]

        # Procesar detecciones
        detecciones = []
        for pred in resultados.predictions:
            detecciones.append({
                'numero': pred.class_name,
                'confianza': round(pred.confidence, 3),
                'bbox': {
                    'x': int(pred.x),
                    'y': int(pred.y),
                    'width': int(pred.width),
                    'height': int(pred.height)
                }
            })

        # Visualizar con supervision
        imagen_anotada = self._visualizar_detecciones(imagen, resultados)

        # Guardar en log CSV
        self._guardar_en_log(detecciones)

        return imagen_anotada, detecciones

    def _visualizar_detecciones(self, imagen: np.ndarray, resultados) -> np.ndarray:
        """Dibuja bounding boxes y etiquetas con supervision"""
        try:
            import supervision as sv

            # Convertir resultados a formato supervision
            detections = sv.Detections.from_inference(resultados)

            # Configurar anotadores
            box_annotator = sv.BoxAnnotator(
                thickness=3,
                color=sv.Color.from_hex("#00FF00")
            )

            label_annotator = sv.LabelAnnotator(
                text_scale=1.2,
                text_thickness=2,
                text_color=sv.Color.WHITE,
                color=sv.Color.from_hex("#00FF00")
            )

            # Generar etiquetas con clase y confianza
            labels = [
                f"{pred.class_name} ({pred.confidence:.2f})"
                for pred in resultados.predictions
            ]

            # Anotar imagen
            imagen_anotada = box_annotator.annotate(
                scene=imagen.copy(),
                detections=detections
            )
            imagen_anotada = label_annotator.annotate(
                scene=imagen_anotada,
                detections=detections,
                labels=labels
            )

            return imagen_anotada

        except Exception as e:
            print(f"[ADVERTENCIA] Error en visualizacion: {e}")
            return imagen

    def _guardar_en_log(self, detecciones: List[Dict]):
        """Añade detecciones al archivo CSV de log"""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        with open(self.csv_log, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            for det in detecciones:
                writer.writerow([
                    timestamp,
                    det['numero'],
                    det['confianza'],
                    'gradio_upload'
                ])

    def calcular_estadisticas(self, detecciones: List[Dict]) -> Dict:
        """Calcula estadisticas de las detecciones"""
        if not detecciones:
            return {
                'total': 0,
                'confianza_promedio': 0.0,
                'confianza_max': 0.0,
                'confianza_min': 0.0
            }

        confianzas = [d['confianza'] for d in detecciones]

        return {
            'total': len(detecciones),
            'confianza_promedio': round(np.mean(confianzas), 3),
            'confianza_max': round(max(confianzas), 3),
            'confianza_min': round(min(confianzas), 3)
        }

    def exportar_csv(self, detecciones: List[Dict], filename: str = None) -> str:
        """Exporta detecciones actuales a CSV"""
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"detecciones_{timestamp}.csv"

        filepath = self.output_dir / filename

        with open(filepath, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['numero', 'confianza', 'bbox'])
            writer.writeheader()
            writer.writerows(detecciones)

        return str(filepath)


print("[OK] Clase JerseyAnalyzer definida")

In [ ]:
# CELDA 5: Interfaz Gradio
# Define la interfaz web para deteccion de dorsales

def crear_interfaz_gradio(analyzer: JerseyAnalyzer):
    """Crea interfaz Gradio profesional con todas las funcionalidades"""

    def analizar_imagen(imagen, confianza_min):
        """Procesa imagen y retorna resultados"""
        if imagen is None:
            return None, "No se cargo ninguna imagen", None

        # Convertir a numpy array RGB
        if isinstance(imagen, Image.Image):
            imagen = np.array(imagen)

        # Detectar numeros
        imagen_anotada, detecciones = analyzer.detectar_numeros(
            imagen,
            confianza_min=confianza_min
        )

        # Calcular estadisticas
        stats = analyzer.calcular_estadisticas(detecciones)

        # Formatear resultados
        texto_stats = f"""
ESTADISTICAS DE DETECCION:
- Total de numeros detectados: {stats['total']}
- Confianza promedio: {stats['confianza_promedio']:.3f}
- Confianza maxima: {stats['confianza_max']:.3f}
- Confianza minima: {stats['confianza_min']:.3f}
        """

        # Formatear tabla de detecciones
        tabla_detecciones = [
            [d['numero'], f"{d['confianza']:.3f}"]
            for d in detecciones
        ]

        return imagen_anotada, texto_stats, tabla_detecciones

    def limpiar_todo():
        """Limpia todos los campos"""
        return None, "", None

    def exportar_resultados_csv(detecciones_tabla):
        """Exporta tabla actual a CSV"""
        if not detecciones_tabla:
            return "No hay detecciones para exportar"

        detecciones = [
            {
                'numero': row[0],
                'confianza': float(row[1]),
                'bbox': {}
            }
            for row in detecciones_tabla
        ]

        filepath = analyzer.exportar_csv(detecciones)
        return f"Exportado a: {filepath}"

    # Crear interfaz con Blocks
    with gr.Blocks(
        title="Basketball Jersey Numbers OCR",
        theme=gr.themes.Soft()
    ) as demo:

        gr.Markdown("# Basketball Jersey Numbers OCR")
        gr.Markdown("Deteccion de numeros en camisetas de baloncesto - Inferencia Local GPU")

        with gr.Row():
            with gr.Column(scale=1):
                imagen_entrada = gr.Image(
                    label="Imagen de entrada",
                    type="numpy",
                    sources=["upload", "webcam"]
                )

                confianza_slider = gr.Slider(
                    minimum=0.1,
                    maximum=0.9,
                    value=0.4,
                    step=0.05,
                    label="Confianza minima"
                )

                with gr.Row():
                    btn_analizar = gr.Button("Analizar", variant="primary")
                    btn_limpiar = gr.Button("Limpiar")

            with gr.Column(scale=1):
                imagen_salida = gr.Image(
                    label="Detecciones",
                    type="numpy"
                )

                texto_stats = gr.Textbox(
                    label="Estadisticas",
                    lines=6,
                    interactive=False
                )

        gr.Markdown("### Historial de Detecciones")

        tabla_detecciones = gr.Dataframe(
            headers=["Numero", "Confianza"],
            label="Resultados",
            interactive=False
        )

        with gr.Row():
            btn_exportar = gr.Button("Exportar a CSV")
            texto_exportar = gr.Textbox(
                label="Estado de exportacion",
                interactive=False
            )

        # Conectar eventos
        btn_analizar.click(
            fn=analizar_imagen,
            inputs=[imagen_entrada, confianza_slider],
            outputs=[imagen_salida, texto_stats, tabla_detecciones]
        )

        btn_limpiar.click(
            fn=limpiar_todo,
            outputs=[imagen_entrada, texto_stats, tabla_detecciones]
        )

        btn_exportar.click(
            fn=exportar_resultados_csv,
            inputs=[tabla_detecciones],
            outputs=[texto_exportar]
        )

    return demo


print("[OK] Funcion de interfaz Gradio definida")

In [ ]:
# CELDA 6: Ejecucion Principal
# IMPORTANTE: Ingresa tu API key de Roboflow antes de ejecutar

# Configuracion
ROBOFLOW_API_KEY = ""  # INGRESA TU API KEY AQUI

if not ROBOFLOW_API_KEY:
    print("[ERROR] Debes ingresar tu Roboflow API key en la variable ROBOFLOW_API_KEY")
    print("\nInstrucciones:")
    print("1. Ve a https://app.roboflow.com")
    print("2. Settings > API Keys")
    print("3. Copia tu Private API Key")
    print("4. Pegala en la variable ROBOFLOW_API_KEY arriba")
else:
    print("=" * 70)
    print("BASKETBALL JERSEY NUMBERS OCR - INFERENCIA LOCAL")
    print("=" * 70)

    # Inicializar analizador
    print("\nInicializando analizador...")
    analyzer = JerseyAnalyzer(api_key=ROBOFLOW_API_KEY)

    # Crear y lanzar interfaz Gradio
    print("\nLanzando interfaz Gradio...")
    print("Se generara una URL publica que puedes abrir en cualquier navegador")
    
    demo = crear_interfaz_gradio(analyzer)
    
    demo.launch(
        share=True,  # Genera URL publica
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True
    )

## Instrucciones de Uso

### 1. Ejecutar Celdas en Orden
1. Celda 1: Instalacion de dependencias (2-3 minutos)
2. Celda 2: Verificacion de GPU
3. Celda 3: Importaciones
4. Celda 4: Clase JerseyAnalyzer
5. Celda 5: Interfaz Gradio
6. Celda 6: Ejecucion (ingresa tu API key primero)

### 2. Obtener API Key de Roboflow
1. Ir a https://app.roboflow.com
2. Settings > API Keys
3. Copiar Private API Key
4. Pegar en variable `ROBOFLOW_API_KEY` de la Celda 6

### 3. Usar la Interfaz
- Subir imagen de camiseta de baloncesto
- Ajustar slider de confianza minima (0.1 - 0.9)
- Hacer clic en "Analizar"
- Ver resultados con bounding boxes
- Exportar a CSV si es necesario

### 4. Archivos Generados
- `/content/jersey_log.csv` - Historial completo de detecciones
- `/content/outputs/detections/` - Exportaciones CSV individuales

### 5. Notas Importantes
- La API key solo se usa para descargar el modelo (primera vez)
- Todas las inferencias son locales en GPU T4 (costo cero)
- El modelo se cachea, la segunda ejecucion es instantanea
- La sesion de Colab expira tras 12 horas de inactividad